In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras import optimizers
import os.path

C:\Users\anton\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# This part simply illustrate the unsatisfactory behavior of the inception validation accuracy
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# dimensions of our images.
# Inception input size
img_width, img_height = 299, 299
train_data_dir = './train'
validation_data_dir = './valid'

# Misc.
nb_train_samples = 192171
nb_validation_samples = 6309
top_epochs = 400
batch_size = 128

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(128, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
opt = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

top_layers_checkpoint_path = 'InceptionLast_cp.h5'

if os.path.exists(top_layers_checkpoint_path):
    model.load_weights(top_layers_checkpoint_path)
    print ("Checkpoint '" + top_layers_checkpoint_path + "' loaded.")

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=45,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

#Save the model after every epoch.
mc_top = ModelCheckpoint(top_layers_checkpoint_path,
                         monitor='val_acc', 
                         verbose=0,
                         save_best_only=True, 
                         save_weights_only=False, 
                         mode='auto', 
                         period=1)

# train the model on the new data for a few epochs
# model.fit_generator(...)
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=top_epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples // batch_size,
    callbacks=[mc_top])

C:\Users\anton\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 192171 images belonging to 128 classes.
Found 6309 images belonging to 128 classes.


C:\Users\anton\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:79: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\anton\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:79: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., callbacks=[<keras.ca..., epochs=400, validation_steps=49, steps_per_epoch=11, validation_data=<keras.pre...)`


Epoch 1/400
11/11 [==============================] - 209s 19s/step - loss: 4.8177 - acc: 0.0256 - val_loss: 4.8538 - val_acc: 0.0190
Epoch 2/400
11/11 [==============================] - 152s 14s/step - loss: 4.6634 - acc: 0.0554 - val_loss: 4.7721 - val_acc: 0.0587
Epoch 3/400
11/11 [==============================] - 150s 14s/step - loss: 4.5495 - acc: 0.0767 - val_loss: 4.6400 - val_acc: 0.0864
Epoch 4/400
11/11 [==============================] - 150s 14s/step - loss: 4.4289 - acc: 0.1165 - val_loss: 4.4941 - val_acc: 0.1192
Epoch 5/400
11/11 [==============================] - 150s 14s/step - loss: 4.3083 - acc: 0.1214 - val_loss: 4.3630 - val_acc: 0.1205
Epoch 6/400
11/11 [==============================] - 150s 14s/step - loss: 4.1624 - acc: 0.1811 - val_loss: 4.2114 - val_acc: 0.1618
Epoch 7/400
11/11 [==============================] - 150s 14s/step - loss: 4.0767 - acc: 0.1690 - val_loss: 4.0703 - val_acc: 0.1877
Epoch 8/400
11/11 [==============================] - 150s 14s/step - 

11/11 [==============================] - 150s 14s/step - loss: 1.8020 - acc: 0.5369 - val_loss: 1.6531 - val_acc: 0.5366
Epoch 124/400
11/11 [==============================] - 150s 14s/step - loss: 1.7091 - acc: 0.5320 - val_loss: 1.6695 - val_acc: 0.5237
Epoch 125/400
11/11 [==============================] - 150s 14s/step - loss: 1.6741 - acc: 0.5724 - val_loss: 1.6494 - val_acc: 0.5318
Epoch 126/400
11/11 [==============================] - 150s 14s/step - loss: 1.6767 - acc: 0.5625 - val_loss: 1.6727 - val_acc: 0.5329
Epoch 127/400
11/11 [==============================] - 150s 14s/step - loss: 1.6468 - acc: 0.5732 - val_loss: 1.6763 - val_acc: 0.5360
Epoch 128/400
11/11 [==============================] - 150s 14s/step - loss: 1.6895 - acc: 0.5597 - val_loss: 1.6546 - val_acc: 0.5394
Epoch 129/400
11/11 [==============================] - 150s 14s/step - loss: 1.7283 - acc: 0.5398 - val_loss: 1.6257 - val_acc: 0.5509
Epoch 130/400
11/11 [==============================] - 150s 14s/step 

Epoch 184/400
11/11 [==============================] - 150s 14s/step - loss: 1.5945 - acc: 0.5653 - val_loss: 1.5532 - val_acc: 0.5643
Epoch 185/400
11/11 [==============================] - 150s 14s/step - loss: 1.6268 - acc: 0.5568 - val_loss: 1.5689 - val_acc: 0.5604
Epoch 186/400
11/11 [==============================] - 150s 14s/step - loss: 1.5713 - acc: 0.5788 - val_loss: 1.5541 - val_acc: 0.5591
Epoch 187/400
11/11 [==============================] - 150s 14s/step - loss: 1.5533 - acc: 0.5866 - val_loss: 1.6147 - val_acc: 0.5454
Epoch 188/400
11/11 [==============================] - 150s 14s/step - loss: 1.6085 - acc: 0.5859 - val_loss: 1.5744 - val_acc: 0.5565
Epoch 189/400
11/11 [==============================] - 150s 14s/step - loss: 1.5120 - acc: 0.5838 - val_loss: 1.5700 - val_acc: 0.5549
Epoch 190/400
11/11 [==============================] - 151s 14s/step - loss: 1.5732 - acc: 0.5881 - val_loss: 1.6006 - val_acc: 0.5525
Epoch 191/400
11/11 [==============================] - 

Epoch 245/400
11/11 [==============================] - 150s 14s/step - loss: 1.5725 - acc: 0.5824 - val_loss: 1.5435 - val_acc: 0.5608
Epoch 246/400
11/11 [==============================] - 151s 14s/step - loss: 1.5119 - acc: 0.5945 - val_loss: 1.5417 - val_acc: 0.5650
Epoch 247/400
11/11 [==============================] - 151s 14s/step - loss: 1.4507 - acc: 0.6087 - val_loss: 1.5269 - val_acc: 0.5693
Epoch 248/400
11/11 [==============================] - 150s 14s/step - loss: 1.4953 - acc: 0.5959 - val_loss: 1.5135 - val_acc: 0.5751
Epoch 249/400
11/11 [==============================] - 150s 14s/step - loss: 1.5344 - acc: 0.5788 - val_loss: 1.5361 - val_acc: 0.5599
Epoch 250/400
11/11 [==============================] - 149s 14s/step - loss: 1.3975 - acc: 0.6286 - val_loss: 1.5198 - val_acc: 0.5672
Epoch 251/400
11/11 [==============================] - 151s 14s/step - loss: 1.4700 - acc: 0.6101 - val_loss: 1.4952 - val_acc: 0.5721
Epoch 252/400
11/11 [==============================] - 

Epoch 306/400
11/11 [==============================] - 151s 14s/step - loss: 1.4319 - acc: 0.6264 - val_loss: 1.4694 - val_acc: 0.5870
Epoch 307/400
11/11 [==============================] - 150s 14s/step - loss: 1.4270 - acc: 0.6051 - val_loss: 1.5344 - val_acc: 0.5650
Epoch 308/400
11/11 [==============================] - 150s 14s/step - loss: 1.4902 - acc: 0.5881 - val_loss: 1.5161 - val_acc: 0.5782
Epoch 309/400
11/11 [==============================] - 150s 14s/step - loss: 1.4285 - acc: 0.6044 - val_loss: 1.5165 - val_acc: 0.5709
Epoch 310/400
11/11 [==============================] - 150s 14s/step - loss: 1.4929 - acc: 0.5945 - val_loss: 1.5136 - val_acc: 0.5865
Epoch 311/400
11/11 [==============================] - 150s 14s/step - loss: 1.5191 - acc: 0.5902 - val_loss: 1.4948 - val_acc: 0.5784
Epoch 312/400
11/11 [==============================] - 150s 14s/step - loss: 1.5112 - acc: 0.5952 - val_loss: 1.5193 - val_acc: 0.5709
Epoch 313/400
11/11 [==============================] - 

Epoch 367/400
11/11 [==============================] - 150s 14s/step - loss: 1.4917 - acc: 0.5973 - val_loss: 1.4736 - val_acc: 0.5871
Epoch 368/400
11/11 [==============================] - 149s 14s/step - loss: 1.4605 - acc: 0.6080 - val_loss: 1.4321 - val_acc: 0.5913
Epoch 369/400
11/11 [==============================] - 150s 14s/step - loss: 1.4253 - acc: 0.6236 - val_loss: 1.4652 - val_acc: 0.5794
Epoch 370/400
11/11 [==============================] - 150s 14s/step - loss: 1.3914 - acc: 0.6229 - val_loss: 1.4830 - val_acc: 0.5803
Epoch 371/400
11/11 [==============================] - 150s 14s/step - loss: 1.4224 - acc: 0.6030 - val_loss: 1.4365 - val_acc: 0.5897
Epoch 372/400
11/11 [==============================] - 150s 14s/step - loss: 1.4511 - acc: 0.6122 - val_loss: 1.4587 - val_acc: 0.5855
Epoch 373/400
11/11 [==============================] - 150s 14s/step - loss: 1.4277 - acc: 0.6271 - val_loss: 1.4542 - val_acc: 0.5751
Epoch 374/400
10/11 [==========================>...] - 